<a href="https://colab.research.google.com/github/skyshine460/GHG_Direct_Emissions/blob/main/ref_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pygeohash

  Preparing metadata (setup.py) ... done
  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6153 sha256=4e1cb978bb724216893100d48fcf1f929447c024e180b4c82e43a3eb0ee0dda3
  Stored in directory: /root/.cache/pip/wheels/28/ec/b6/beadf7295a623f528507691fb0d471b50d064ae9bbad420b8f
Successfully built pygeohash


In [2]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.6 MB/s eta 0:00:00


In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib
import math

import pygeohash as pgh

from concurrent.futures import ThreadPoolExecutor

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder

import gc
gc.collect()

import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# 予測モデルを訓練するためのデータセット
train_df = pd.read_csv('/content/drive/MyDrive/GHGの排出量の予測にチャレンジしよう！（SMBC Group GREEN×DATA Challenge 2024）/提供データ/train.csv', index_col=0)

# 予測モデルに推論（予測)させるデータセット
test_df = pd.read_csv('/content/drive/MyDrive/GHGの排出量の予測にチャレンジしよう！（SMBC Group GREEN×DATA Challenge 2024）/提供データ/test.csv', index_col=0)

In [6]:
# del train_df['Unnamed: 0'], test_df['Unnamed: 0']
del train_df['FacilityName'], test_df['FacilityName']
del train_df['LocationAddress'], test_df['LocationAddress']
del train_df['ZIP'], test_df['ZIP']
del train_df['IndustryType'], test_df['IndustryType']
del train_df['SecondPrimaryNAICS'], test_df['SecondPrimaryNAICS']

In [7]:
two_digit_map     = {11: 'Agriculture, Forestry, Fishing and Hunting',
                    21: 'Mining, Quarrying, and Oil and Gas Extraction',
                    22: 'Utilities',
                    23: 'Construction',
                    31: 'Manufacturing',
                    32: 'Manufacturing',
                    33: 'Manufacturing',
                    42: 'Wholesale Trade',
                    44: 'Retail Trade',
                    45: 'Retail Trade',
                    48: 'Transportation and Warehousing',
                    49: 'Transportation and Warehousing',
                    51: 'Information',
                    52: 'Finance and Insurance',
                    53: 'Real Estate and Rental and Leasing',
                    54: 'Professional, Scientific, and Technical Services',
                    55: 'Management of Companies and Enterprises',
                    56: 'Administrative and Support and Waste Management and Remediation Services',
                    61: 'Educational Services',
                    62: 'Health Care and Social Assistance',
                    71: 'Arts, Entertainment, and Recreation',
                    72: 'Accommodation and Food Services',
                    81: 'Other Services (except Public Administration)',
                    92: 'Public Administration'}

In [8]:
train_df['first_two_digit_primary_naics'] = train_df['PrimaryNAICS'].apply(lambda z: str(z)[:2]).astype(int)
test_df['first_two_digit_primary_naics']  = test_df['PrimaryNAICS'].apply(lambda z: str(z)[:2]).astype(int)

train_df['Economic_Sector']               = train_df['first_two_digit_primary_naics'].map(two_digit_map)
test_df['Economic_Sector']                = test_df['first_two_digit_primary_naics'].map(two_digit_map)

del train_df['first_two_digit_primary_naics'], test_df['first_two_digit_primary_naics']

econ_sector_train                         = train_df['Economic_Sector'].values
econ_sector_test                          = test_df['Economic_Sector'].values

In [9]:
train_distance = pd.read_csv('/content/drive/MyDrive/GHGの排出量の予測にチャレンジしよう！（SMBC Group GREEN×DATA Challenge 2024）/特徴量/train_distance_5nbrs_.csv')
test_distance = pd.read_csv('/content/drive/MyDrive/GHGの排出量の予測にチャレンジしよう！（SMBC Group GREEN×DATA Challenge 2024）/特徴量/test_distance_5nbrs_.csv')

In [10]:
# def haversine(lat1, lon1, lat2, lon2):
#     # Convert latitude and longitude from degrees to radians
#     lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

#     # Haversine formula
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
#     c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

#     # Radius of Earth in kilometers (mean radius)
#     R = 6371.0

#     # Calculate the distance
#     distance = R * c
#     return distance


In [11]:
# def calculate_distance(lat1, lon1, lat2, lon2):
#     return haversine(lat1, lon1, lat2, lon2)

# def get_nearest_distance(lat1, lon1, econ_sector_ref, neighbours=5, train_point=False):
#     # Extract the required columns once to NumPy arrays
#     latitudes = train_df['Latitude'].values
#     longitudes = train_df['Longitude'].values
#     ghg_emissions = train_df['GHG_Direct_Emissions_14_in_metric_tons'].values
#     econ_sectors = train_df['Economic_Sector'].values

#     # Filter NaN emissions directly
#     valid_indices = ~np.isnan(ghg_emissions)

#     latitudes = latitudes[valid_indices]
#     longitudes = longitudes[valid_indices]
#     ghg_emissions = ghg_emissions[valid_indices]
#     econ_sectors = econ_sectors[valid_indices]

#     # Parallelize distance calculation using ThreadPoolExecutor
#     with ThreadPoolExecutor() as executor:
#         distances = list(executor.map(calculate_distance,
#                                      [lat1]*len(latitudes),
#                                      [lon1]*len(longitudes),
#                                      latitudes,
#                                      longitudes))

#     # Combine the results into a DataFrame
#     near_df = pd.DataFrame({
#         'Distance': distances,
#         'GHG_emission_14': ghg_emissions,
#         'Economic_Sector': econ_sectors
#     })


#     if train_point:
#         near_df.sort_values(by='Distance', inplace=True)
#         near_df = near_df.dropna()
#         near_df = near_df.iloc[1:].reset_index(drop=True)
#     else:
#         # Sort distances by 'Distance'
#         near_df.sort_values(by='Distance', inplace=True)
#         near_df = near_df.dropna()

#     # Filter by economic sector
#     nearest_locations_econ_sector = near_df[near_df['Economic_Sector'] == econ_sector_ref]

#     # Get the top N nearest for both economic sector and overall
#     sub_near_econ = nearest_locations_econ_sector.head(neighbours)
#     sub_nearest_locations = near_df.head(neighbours)

#     # Compute weighted averages
#     econ_weighted_average = (sub_near_econ['GHG_emission_14'] / np.where(sub_near_econ['Distance'] == 0, 1, sub_near_econ['Distance']) ).sum()
#     near_weighted_average = (sub_nearest_locations['GHG_emission_14'] / np.where(sub_nearest_locations['Distance']==0,1,sub_nearest_locations['Distance'])).sum()

#     # Compute regular averages
#     econ_average = sub_near_econ['GHG_emission_14'].mean()
#     near_average = sub_nearest_locations['GHG_emission_14'].mean()

#     return [econ_weighted_average, econ_average, near_weighted_average, near_average]

In [12]:
# neighbours = 5

# file_path = f"test_distance_{neighbours}nbrs.csv"
# if os.path.exists(file_path):
#     print(f"{file_path} exists. ")
#     test_distance = pd.read_csv(f"test_distance_{neighbours}nbrs.csv")
# else:
#     output = []
#     for index in tqdm(range(test_df.shape[0])):

#         lat1 = test_df.iloc[index]['Latitude']
#         lon1 = test_df.iloc[index]['Longitude']
#         econ_sector_ref = test_df.iloc[index]['Economic_Sector']
#         x = get_nearest_distance(lat1,lon1,econ_sector_ref,neighbours=neighbours,train_point=False)
#         output.append(x)

#     test_distance = pd.DataFrame(output,columns = ['Economy_Sector_Weighted_Avg','Economic_Sector_Average','Nearest_Weighted_Average','Nearest_Average'])

#     test_distance.to_csv(f'test_distance_{neighbours}nbrs.csv',index=False)


# file_path = f"train_distance_{neighbours}nbrs.csv"
# if os.path.exists(file_path):
#     print(f"{file_path} exists. ")
#     train_distance = pd.read_csv(f'train_distance_{neighbours}nbrs.csv')
# else:
#     output = []
#     for index in tqdm(range(train_df.shape[0])):
#         lat1 = train_df.iloc[index]['Latitude']
#         lon1 = train_df.iloc[index]['Longitude']
#         econ_sector_ref = train_df.iloc[index]['Economic_Sector']
#         x = get_nearest_distance(lat1,lon1,econ_sector_ref,neighbours=neighbours, train_point=True)
#         output.append(x)

#     train_distance = pd.DataFrame(output,columns = ['Economy_Sector_Weighted_Avg','Economic_Sector_Average','Nearest_Weighted_Average','Nearest_Average'])
#     train_distance.to_csv(f'train_distance_{neighbours}nbrs.csv',index=False)



train_df = pd.concat((train_df,train_distance),axis=1)
test_df  = pd.concat((test_df,test_distance),axis=1)

In [13]:
def create_features1(df):


    # Year-over-year difference
    df['TRI_Air_Emissions_YoY_Change_11'] = df['TRI_Air_Emissions_11_in_lbs'] - df['TRI_Air_Emissions_10_in_lbs']
    df['TRI_Air_Emissions_YoY_Change_12'] = df['TRI_Air_Emissions_12_in_lbs'] - df['TRI_Air_Emissions_11_in_lbs']
    df['TRI_Air_Emissions_YoY_Change_13'] = df['TRI_Air_Emissions_13_in_lbs'] - df['TRI_Air_Emissions_12_in_lbs']

   # Year-over-year growth rate with NaN protection
    df['TRI_Air_Emissions_Growth_Rate_11'] = np.where(
        df['TRI_Air_Emissions_10_in_lbs'].notna() & (df['TRI_Air_Emissions_10_in_lbs'] != 0),
        (df['TRI_Air_Emissions_11_in_lbs'] - df['TRI_Air_Emissions_10_in_lbs']) / df['TRI_Air_Emissions_10_in_lbs'],
        np.nan
    )

    df['TRI_Air_Emissions_Growth_Rate_12'] = np.where(
        df['TRI_Air_Emissions_11_in_lbs'].notna() & (df['TRI_Air_Emissions_11_in_lbs'] != 0),
        (df['TRI_Air_Emissions_12_in_lbs'] - df['TRI_Air_Emissions_11_in_lbs']) / df['TRI_Air_Emissions_11_in_lbs'],
        np.nan
    )

    df['TRI_Air_Emissions_Growth_Rate_13'] = np.where(
        df['TRI_Air_Emissions_12_in_lbs'].notna() & (df['TRI_Air_Emissions_12_in_lbs'] != 0),
        (df['TRI_Air_Emissions_13_in_lbs'] - df['TRI_Air_Emissions_12_in_lbs']) / df['TRI_Air_Emissions_12_in_lbs'],
        np.nan
    )

    return df

train_df      = create_features1(train_df)
test_df       = create_features1(test_df)
new_features1 = ['TRI_Air_Emissions_YoY_Change_11','TRI_Air_Emissions_YoY_Change_12','TRI_Air_Emissions_YoY_Change_13',
                 'TRI_Air_Emissions_Growth_Rate_11','TRI_Air_Emissions_Growth_Rate_12','TRI_Air_Emissions_Growth_Rate_13']


In [14]:
def create_features2(df):


    # Year-over-year difference
    df['GHG_Direct_Emissions_YoY_Change_11'] = df['GHG_Direct_Emissions_11_in_metric_tons'] - df['GHG_Direct_Emissions_10_in_metric_tons']
    df['GHG_Direct_Emissions_YoY_Change_12'] = df['GHG_Direct_Emissions_12_in_metric_tons'] - df['GHG_Direct_Emissions_11_in_metric_tons']
    df['GHG_Direct_Emissions_YoY_Change_13'] = df['GHG_Direct_Emissions_13_in_metric_tons'] - df['GHG_Direct_Emissions_12_in_metric_tons']

    # Year-over-year growth rate for GHG Direct Emissions with NaN protection
    df['GHG_Direct_Emissions_Growth_Rate_11'] = np.where(
        df['GHG_Direct_Emissions_10_in_metric_tons'].notna() & (df['GHG_Direct_Emissions_10_in_metric_tons'] != 0),
        (df['GHG_Direct_Emissions_11_in_metric_tons'] - df['GHG_Direct_Emissions_10_in_metric_tons']) / df['GHG_Direct_Emissions_10_in_metric_tons'],
        np.nan
    )

    df['GHG_Direct_Emissions_Growth_Rate_12'] = np.where(
        df['GHG_Direct_Emissions_11_in_metric_tons'].notna() & (df['GHG_Direct_Emissions_11_in_metric_tons'] != 0),
        (df['GHG_Direct_Emissions_12_in_metric_tons'] - df['GHG_Direct_Emissions_11_in_metric_tons']) / df['GHG_Direct_Emissions_11_in_metric_tons'],
        np.nan
    )

    df['GHG_Direct_Emissions_Growth_Rate_13'] = np.where(
        df['GHG_Direct_Emissions_12_in_metric_tons'].notna() & (df['GHG_Direct_Emissions_12_in_metric_tons'] != 0),
        (df['GHG_Direct_Emissions_13_in_metric_tons'] - df['GHG_Direct_Emissions_12_in_metric_tons']) / df['GHG_Direct_Emissions_12_in_metric_tons'],
        np.nan
    )

    return df


new_features2 = ['GHG_Direct_Emissions_YoY_Change_11','GHG_Direct_Emissions_YoY_Change_12','GHG_Direct_Emissions_YoY_Change_13',
                 'GHG_Direct_Emissions_Growth_Rate_11','GHG_Direct_Emissions_Growth_Rate_12','GHG_Direct_Emissions_Growth_Rate_13'
                 ]
train_df      = create_features2(train_df)
test_df       = create_features2(test_df)

In [15]:
def create_features3(df):
    df['TRI_to_GHG_Ratio_10'] = np.where(df['GHG_Direct_Emissions_10_in_metric_tons'].notna(),
                                     df['TRI_Air_Emissions_10_in_lbs'] / df['GHG_Direct_Emissions_10_in_metric_tons'],
                                     np.nan)

    df['TRI_to_GHG_Ratio_11'] = np.where(df['GHG_Direct_Emissions_11_in_metric_tons'].notna(),
                                        df['TRI_Air_Emissions_11_in_lbs'] / df['GHG_Direct_Emissions_11_in_metric_tons'],
                                        np.nan)

    df['TRI_to_GHG_Ratio_12'] = np.where(df['GHG_Direct_Emissions_12_in_metric_tons'].notna(),
                                        df['TRI_Air_Emissions_12_in_lbs'] / df['GHG_Direct_Emissions_12_in_metric_tons'],
                                        np.nan)

    df['TRI_to_GHG_Ratio_13'] = np.where(df['GHG_Direct_Emissions_13_in_metric_tons'].notna(),
                                        df['TRI_Air_Emissions_13_in_lbs'] / df['GHG_Direct_Emissions_13_in_metric_tons'],
                                        np.nan)
    return df



train_df      = create_features3(train_df)
test_df       = create_features3(test_df)
new_features3 = ['TRI_to_GHG_Ratio_10','TRI_to_GHG_Ratio_11','TRI_to_GHG_Ratio_12','TRI_to_GHG_Ratio_13']

In [16]:
summary_df = train_df.groupby(['Economic_Sector', 'State']).agg({'GHG_Direct_Emissions_14_in_metric_tons': ['mean', 'median', 'max', 'min','count']})

summary_df.columns = [
                        'GHG_Direct_Emissions_14_in_metric_tons_mean',
                        'GHG_Direct_Emissions_14_in_metric_tons_median',
                        'GHG_Direct_Emissions_14_in_metric_tons_max',
                        'GHG_Direct_Emissions_14_in_metric_tons_min',
                        'GHG_Direct_Emissions_14_in_metric_tons_count'
                     ]

summary_df = summary_df.reset_index()

train_df = train_df.merge(summary_df, on=['Economic_Sector', 'State'], how='left')
test_df  = test_df.merge(summary_df,  on=['Economic_Sector', 'State'], how='left')


In [17]:
merged_df = pd.concat((train_df,test_df),axis=0)
for cols in ['City','State','County','FIPScode','PrimaryNAICS','Economic_Sector']:
    le              = LabelEncoder()
    merged_df[cols] = le.fit_transform(merged_df[cols].values.reshape(-1,1))

train_df = merged_df.iloc[:train_df.shape[0],:]
test_df  = merged_df.iloc[train_df.shape[0]:,:]
train_df.shape,test_df.shape

((4655, 42), (5016, 42))

In [18]:
numerical_columns = [
                    'TRI_Air_Emissions_10_in_lbs', 'TRI_Air_Emissions_11_in_lbs',
                    'TRI_Air_Emissions_12_in_lbs', 'TRI_Air_Emissions_13_in_lbs','PrimaryNAICS',
                    'GHG_Direct_Emissions_10_in_metric_tons', 'GHG_Direct_Emissions_11_in_metric_tons',
                    'GHG_Direct_Emissions_12_in_metric_tons', 'GHG_Direct_Emissions_13_in_metric_tons',
                    ]
lat_lon_columns   = ['Latitude','Longitude']
target_columns    = ['GHG_Direct_Emissions_14_in_metric_tons']
categorical_columns = ['City','State','County','FIPScode','PrimaryNAICS','Economic_Sector']
train_aggregations  =  [
                        'GHG_Direct_Emissions_14_in_metric_tons_mean',
                        'GHG_Direct_Emissions_14_in_metric_tons_median',
                        'GHG_Direct_Emissions_14_in_metric_tons_max',
                        'GHG_Direct_Emissions_14_in_metric_tons_min',
                        'GHG_Direct_Emissions_14_in_metric_tons_count'
                     ]
new_features1  = ['TRI_Air_Emissions_YoY_Change_11','TRI_Air_Emissions_YoY_Change_12','TRI_Air_Emissions_YoY_Change_13',
                    'TRI_Air_Emissions_Growth_Rate_11','TRI_Air_Emissions_Growth_Rate_12','TRI_Air_Emissions_Growth_Rate_13']

new_features2   = ['GHG_Direct_Emissions_YoY_Change_11','GHG_Direct_Emissions_YoY_Change_12','GHG_Direct_Emissions_YoY_Change_13',
                  'GHG_Direct_Emissions_Growth_Rate_11','GHG_Direct_Emissions_Growth_Rate_12','GHG_Direct_Emissions_Growth_Rate_13'
                  ]
new_features3   = ['TRI_to_GHG_Ratio_10','TRI_to_GHG_Ratio_11','TRI_to_GHG_Ratio_12','TRI_to_GHG_Ratio_13']
neighbour_feats = ['Economy_Sector_Weighted_Avg','Economic_Sector_Average','Nearest_Weighted_Average','Nearest_Average']


train = train_df[numerical_columns+
                 lat_lon_columns+
                #  categorical_columns+
                 new_features1+['Economic_Sector']
                #  new_features2
               #   new_features3+
                #  train_aggregations+
               #   neighbour_feats
                 ].values
test  = test_df[numerical_columns+
                 lat_lon_columns+
                #  categorical_columns+
                 new_features1+['Economic_Sector']
                #  new_features2
               #   new_features3+
                #  train_aggregations+
               #   neighbour_feats
                 ].values
target = train_df[target_columns].values

In [19]:
# def get_models_trained(train,test,target, num_folds):
#     kf               = KFold(n_splits=num_folds, shuffle=True, random_state=13)

#     oof_predictions  = np.zeros(len(train))
#     test_predictions = np.zeros(len(test))


#     for fold, (train_index, valid_index) in enumerate(kf.split(train,target)):

#         X_train, X_valid             = train[train_index], train[valid_index]
#         y_train, y_valid             = target[train_index], target[valid_index]


#         model_dict                   = {}
#         loss_dict                    = {}
#         global valid_pred_dict
#         valid_pred_dict              = {}

#         params                       = {"n_estimators": 626, "max_depth": 3, "random_state":13,
#                                         "min_child_weight": 0.001190123543553736, "learning_rate": 0.010519736270936835,
#                                         "subsample": 0.7304788478701394, "colsample_bylevel": 0.604447278915981,
#                                           "colsample_bytree": 0.7616852136157319, "reg_alpha": 0.115175569924065, "reg_lambda": 0.07155347824929895}
#         model1                       = XGBRegressor(**params)

#         params                       = {"n_estimators": 206, "max_depth": 3, "min_child_weight": 0.002124665025111174, "random_state":13,
#                                         "learning_rate": 0.02528390491004826, "subsample": 0.800800019181945,
#                                         "colsample_bylevel": 0.7264139639244361, "colsample_bytree": 0.7283285945816331,
#                                           "reg_alpha": 0.13673920290025274, "reg_lambda": 0.008614256283329808}
#         model2                       = XGBRegressor(**params)

#         params                       = {"n_estimators": 56,"random_state":13,
#                                          "verbose":-1,}
#         model3                       = LGBMRegressor(**params)



#         _                             = model1.fit(X_train,np.log1p(y_train))
#         valid_preds1                  = np.expm1(model1.predict(X_valid))
#         rmsle1                        = root_mean_squared_log_error(y_valid, valid_preds1)
#         print(f"Fold {fold+1} RMSLE for model1 = {rmsle1}")

#         _                             = model2.fit(X_train,np.log1p(y_train))
#         valid_preds2                  = np.expm1(model2.predict(X_valid))
#         rmsle2                        = root_mean_squared_log_error(y_valid, valid_preds2)
#         print(f"Fold {fold+1} RMSLE for model2 = {rmsle2}")

#         _                             = model3.fit(X_train,np.log1p(y_train))
#         valid_preds3                  = np.expm1(model3.predict(X_valid))
#         rmsle3                        = root_mean_squared_log_error(y_valid, valid_preds3)
#         print(f"Fold {fold+1} RMSLE for model3 = {rmsle3}")



#         loss_dict['model1']           = rmsle1
#         loss_dict['model2']           = rmsle2
#         loss_dict['model3']           = rmsle3


#         model_dict['model1']          = model1
#         model_dict['model2']          = model2
#         model_dict['model3']          = model3


#         valid_pred_dict['model1']     = valid_preds1
#         valid_pred_dict['model2']     = valid_preds2
#         valid_pred_dict['model3']     = valid_preds3



#         valid_preds_mean              = np.mean(list(valid_pred_dict.values()), axis=0)
#         rmsle_mean                    = root_mean_squared_log_error(y_valid, valid_preds_mean)
#         print(f"Fold {fold+1} Average RMSLE = {rmsle_mean}")


#         min_loss_model                = min(loss_dict, key=loss_dict.get)

#         model                         = model_dict[min_loss_model]
#         valid_preds_best_model        = np.expm1(model.predict(X_valid))
#         rmsle_best                    = root_mean_squared_log_error(y_valid, valid_preds_best_model)


#         if rmsle_mean >rmsle_best:
#             print(f"The average RMSLE is {rmsle_mean} while the best RMSLE is {rmsle_best} and we proceed with the model with best RMSLE")
#             model = model_dict[min_loss_model]
#             oof_predictions[valid_index]  = valid_preds_best_model
#             test_preds                    = model.predict(test)
#             test_predictions += (test_preds) / kf.n_splits
#             oof_predictions[valid_index]  = valid_preds_best_model


#         else:
#             print(f"The average RMSLE is {rmsle_mean} while the best RMSLE is {rmsle_best} and we proceed with averaging of all models")
#             output_predictions = []
#             for key,values in model_dict.items():
#                 model       = values
#                 test_preds  = model.predict(test)
#                 _           = output_predictions.append(test_preds)
#             output_preds    = np.mean(output_predictions, axis=0)
#             test_predictions += output_preds/kf.n_splits
#             oof_predictions[valid_index]  = valid_preds_mean







#         gc.collect()

#         print('---------------\n')

#     RMSLE = root_mean_squared_log_error(target, oof_predictions)
#     print(f"OOF RMSLE = {RMSLE}")

#     return oof_predictions,np.expm1(test_predictions)


In [20]:
def get_models_trained(train, test, target, num_folds):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=13)

    oof_predictions = np.zeros(len(train))
    test_predictions = np.zeros(len(test))
    train_predictions = np.zeros(len(train))  # Trainデータ予測の格納用

    for fold, (train_index, valid_index) in enumerate(kf.split(train, target)):
        print(f"------------------------------ fold {fold+1} ------------------------------")

        X_train, X_valid = train[train_index], train[valid_index]
        y_train, y_valid = target[train_index], target[valid_index]

        model_dict = {}
        loss_dict = {}
        valid_pred_dict = {}

        # モデル1
        params = {
            "n_estimators": 626,
            "max_depth": 3,
            "random_state": 13,
            "min_child_weight": 0.001190123543553736,
            "learning_rate": 0.010519736270936835,
            "subsample": 0.7304788478701394,
            "colsample_bylevel": 0.604447278915981,
            "colsample_bytree": 0.7616852136157319,
            "reg_alpha": 0.115175569924065,
            "reg_lambda": 0.07155347824929895,
        }
        model1 = XGBRegressor(**params)

        # モデル2
        params = {
            "n_estimators": 206,
            "max_depth": 3,
            "min_child_weight": 0.002124665025111174,
            "random_state": 13,
            "learning_rate": 0.02528390491004826,
            "subsample": 0.800800019181945,
            "colsample_bylevel": 0.7264139639244361,
            "colsample_bytree": 0.7283285945816331,
            "reg_alpha": 0.13673920290025274,
            "reg_lambda": 0.008614256283329808,
        }
        model2 = XGBRegressor(**params)

        # モデル3
        params = {"n_estimators": 56, "random_state": 13, "verbose": -1}
        model3 = LGBMRegressor(**params)

        # モデル1の学習と予測
        _ = model1.fit(X_train, np.log1p(y_train))
        valid_preds1 = np.expm1(model1.predict(X_valid))
        rmsle1 = root_mean_squared_log_error(y_valid, valid_preds1)
        print(f"Fold {fold+1} RMSLE for model1 = {rmsle1}")

        # モデル2の学習と予測
        _ = model2.fit(X_train, np.log1p(y_train))
        valid_preds2 = np.expm1(model2.predict(X_valid))
        rmsle2 = root_mean_squared_log_error(y_valid, valid_preds2)
        print(f"Fold {fold+1} RMSLE for model2 = {rmsle2}")

        # モデル3の学習と予測
        _ = model3.fit(X_train, np.log1p(y_train))
        valid_preds3 = np.expm1(model3.predict(X_valid))
        rmsle3 = root_mean_squared_log_error(y_valid, valid_preds3)
        print(f"Fold {fold+1} RMSLE for model3 = {rmsle3}")

        # 損失とモデルの記録
        loss_dict["model1"] = rmsle1
        loss_dict["model2"] = rmsle2
        loss_dict["model3"] = rmsle3

        model_dict["model1"] = model1
        model_dict["model2"] = model2
        model_dict["model3"] = model3

        valid_pred_dict["model1"] = valid_preds1
        valid_pred_dict["model2"] = valid_preds2
        valid_pred_dict["model3"] = valid_preds3

        # 平均予測
        valid_preds_mean = np.mean(list(valid_pred_dict.values()), axis=0)
        rmsle_mean = root_mean_squared_log_error(y_valid, valid_preds_mean)
        print(f"Fold {fold+1} Average RMSLE = {rmsle_mean}")

        min_loss_model = min(loss_dict, key=loss_dict.get)

        model = model_dict[min_loss_model]
        valid_preds_best_model = np.expm1(model.predict(X_valid))
        rmsle_best = root_mean_squared_log_error(y_valid, valid_preds_best_model)

        if rmsle_mean > rmsle_best:
            print(f"The average RMSLE is {rmsle_mean} while the best RMSLE is {rmsle_best}. Using the best model.")
            oof_predictions[valid_index] = valid_preds_best_model
        else:
            print(f"The average RMSLE is {rmsle_mean} while the best RMSLE is {rmsle_best}. Averaging all models.")
            oof_predictions[valid_index] = valid_preds_mean

        # 全Trainデータ予測
        for train_idx in train_index:
            train_predictions[train_idx] += valid_preds_best_model.mean() / kf.n_splits

        # テストデータの予測
        test_preds = model.predict(test)
        test_predictions += test_preds / kf.n_splits

        gc.collect()
        print("---------------\n")

    RMSLE = root_mean_squared_log_error(target, oof_predictions)
    print(f"OOF RMSLE = {RMSLE}")
    print(f"Train RMSLE = {root_mean_squared_log_error(target, train_predictions)}")

    return oof_predictions, train_predictions, np.expm1(test_predictions)


In [21]:
# oof_predictions,test_predictions = get_models_trained(train,test,target,30)
oof_predictions, train_predictions, test_predictions = get_models_trained(train, test, target, 20)

------------------------------ fold 1 ------------------------------
Fold 1 RMSLE for model1 = 0.7415377868454196
Fold 1 RMSLE for model2 = 0.7401615326048081
Fold 1 RMSLE for model3 = 0.7588276094861218
Fold 1 Average RMSLE = 0.7435481212060641
The average RMSLE is 0.7435481212060641 while the best RMSLE is 0.7401615326048081. Using the best model.
---------------

------------------------------ fold 2 ------------------------------
Fold 2 RMSLE for model1 = 0.6208200937805237
Fold 2 RMSLE for model2 = 0.6170278922675616
Fold 2 RMSLE for model3 = 0.6456541275864296
Fold 2 Average RMSLE = 0.6232742412669424
The average RMSLE is 0.6232742412669424 while the best RMSLE is 0.6170278922675616. Using the best model.
---------------

------------------------------ fold 3 ------------------------------
Fold 3 RMSLE for model1 = 0.7557214519060115
Fold 3 RMSLE for model2 = 0.7562952177184386
Fold 3 RMSLE for model3 = 0.7637242061317118
Fold 3 Average RMSLE = 0.7550316082904608
The average RMSL

In [22]:
submit  =  pd . read_csv ( 'sample_submission.csv' ,  header = None )
submit [ 1 ]  =  test_predictions
submit . to_csv ( 'submission_26.csv' ,  header = None ,  index = False )

FileNotFoundError: [Errno 2] No such file or directory: 'sample_submission.csv'